# Test Correspondences

In [ ]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import NED_to_ENU, pc_plot_trace
from planeslam.scan import velo_pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'pcs')
PCs = []
#len(os.listdir(pcpath))
#select_idxs = np.arange(0, 1000, 2)
select_idxs = np.arange(0, len(os.listdir(pcpath)), 10)
for i in select_idxs:  
    filename = os.path.join(pcpath, 'pc_'+str(i)+'.npy')
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'poses')
poses = []
for i in select_idxs:  
    filename = os.path.join(posepath, 'pose_'+str(i)+'.npy')
    pose = np.load(filename)
    poses.append(pose)

Extract planes 

In [ ]:
# Plot 2 scans
source_idx = 26
target_idx = 25
P_source = velo_preprocess(PCs[source_idx], poses[source_idx])
P_target = velo_preprocess(PCs[target_idx], poses[target_idx])
source = velo_pc_to_scan(P_source)
target = velo_pc_to_scan(P_target)

source.remove_small_planes(area_thresh=0.1)
source.reduce_inside(p2p_dist_thresh=0.1)
target.remove_small_planes(area_thresh=0.1)
target.reduce_inside(p2p_dist_thresh=0.1)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)
fig.add_trace(pc_plot_trace(P_source), row=1, col=1)

for t in target.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)
fig.add_trace(pc_plot_trace(P_target), row=1, col=2)

fig.update_layout(width=1600, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
fig = go.Figure(data=source.plot_trace()+[pc_plot_trace(P_source)])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target)

print(R_hat)
print(t_hat)

In [ ]:
from planeslam.geometry.util import quat_to_R

R_1 = quat_to_R(poses[target_idx][3:])
R_2 = quat_to_R(poses[source_idx][3:])
R_gt = R_2 @ R_1.T

t_gt = poses[source_idx][:3] - poses[target_idx][:3]
print(R_gt)
print(t_gt)

In [ ]:
source.transform(R_hat, t_hat.flatten())

In [ ]:
fig = go.Figure(data=source.plot_trace()+target.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

[(0, 0), (1, 3), (2, 1), (3, 2), (4, 4), (5, 5), (6, 8), (7, 6), (8, 7), (9, 9), (10, 10), (11, 11)]